In [1]:
import numpy as np
import cv2
import os
import torch as t
import visdom
import time

from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils import data

vis = visdom.Visdom(env = 'grad_FC')



In [2]:
kernel_ = np.array([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])

p = np.zeros((28,3,3))
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,1] = 1; p[0] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,2] = 1; p[1] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,0] = 1; p[2] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,2] = 1; p[3] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,0] = 1; p[4] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,1] = 1; p[5] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,2] = 1; p[6] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[0,2] = 1; p[7] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,0] = 1; p[8] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,2] = 1; p[9] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,0] = 1; p[10] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,1] = 1; p[11] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,2] = 1; p[12] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,0] = 1; p[13] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,2] = 1; p[14] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,0] = 1; p[15] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,1] = 1; p[16] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,2] = 1; p[17] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[1,2] = 1; p[18] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,0] = 1; p[19] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,1] = 1; p[20] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,2] = 1; p[21] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,0] = 1; p[22] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,1] = 1; p[23] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,2] = 1; p[24] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,1] = 1; p[25] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,2] = 1; p[26] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,1] = 1;zeros[2,2] = 1; p[27] = zeros


In [3]:
#网络model
class FC_Net(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim):
        super().__init__()
        self.layer1 = nn.Sequential(
                                    
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_2),
                                    nn.BatchNorm1d(n_hidden_2),
                                    nn.ReLU(True)
                                    )
        self.layer3 = nn.Sequential(
                                    nn.Linear(n_hidden_2,n_hidden_3),
                                    nn.BatchNorm1d(n_hidden_3),
                                    nn.ReLU(True)
                                    )
        self.layer_out = nn.Sequential(
                                    nn.Linear(n_hidden_3,output_dim),
                                    #nn.BatchNorm1d(output_dim),
                                    #nn.ReLU(True)
                                       )

    def forward(self,x):
        #print("x.shape",x.shape)
        x = self.layer1(x)
        #print("x.shape1",x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer_out(x)
        #x = self.softmax(x)
        #print(x)
        return x

In [4]:

class var_dataset(data.Dataset):
    def __init__(self,root,transforms0=transforms.ToTensor()):
        self.dataset0 = ImageFolder(root)
        self.labels = [img[1] for img in self.dataset0.imgs]
        self.transforms0 = transforms0
        
    def __getitem__(self,index):        
        label = self.dataset0[index][1]
        #image = np.asarray(self.dataset0[index][0])         #这一行说明，cv2也是使用的numpy的数据（矩阵）
        image_gaussion = cv2.filter2D(np.asarray(self.dataset0[index][0]),cv2.CV_16S,kernel_)
        img_2grad = np.zeros((28,120,120,3))
        var_array = np.zeros((1,84))
        var_array1 = np.zeros((1,28))
        aver_array = np.zeros((1,84))
        aver_array1 = np.zeros((1,28))
        for i in range(28):
            img_2grad[i] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[i])
            var_array[0,3*i] = img_2grad[i][:,:,0].var()
            var_array[0,3*i+1] = img_2grad[i][:,:,1].var()
            var_array[0,3*i+2] = img_2grad[i][:,:,2].var()
            aver_array[0,3*i] = img_2grad[i][:,:,0].mean()
            aver_array[0,3*i+1] = img_2grad[i][:,:,1].mean()
            aver_array[0,3*i+2] = img_2grad[i][:,:,2].mean()
            var_array1[0,i] = img_2grad[i].var()
            aver_array1[0,i] = img_2grad[i].mean()
        
        var_array_max = var_array.max()
        var_array_min = var_array.min()
        aver_array_max = aver_array.max()
        aver_array_min = aver_array.min()
        var_array_max1 = var_array1.max()
        var_array_min1 = var_array1.min()
        aver_array_max1 = aver_array1.max()
        aver_array_min1 = aver_array1.min()
        
        var_array = np.true_divide((var_array-var_array_min),(var_array_max-var_array_min))
        aver_array = np.true_divide((aver_array-aver_array_min),(aver_array_max-aver_array_min))
        var_array1 = np.true_divide((var_array1-var_array_min1),(var_array_max1-var_array_min1))
        aver_array1 = np.true_divide((aver_array1-aver_array_min1),(aver_array_max1-aver_array_min1))
        
        var_array = np.true_divide((var_array-0.5),0.5) 
        aver_array = np.true_divide((aver_array-0.5),0.5)
        var_array1 = np.true_divide((var_array1-0.5),0.5) 
        aver_array1 = np.true_divide((aver_array1-0.5),0.5)
        result_array = np.hstack((var_array,aver_array,var_array1,aver_array1)).reshape(224)
        result_array = t.Tensor(result_array)
        
        return result_array,label
    
    def __len__(self):
        return len(self.labels)
                                        

In [18]:
#实验的CELL   
#说明，hstack()可以多个矩阵。
a = np.array([[1,-1,1,1]])
b = np.array([[-1,-1,1,-1]])
c = np.array([[-1,-1,0,-1]])
d = np.array([[0,-1,1,-1]])
np.true_divide((a-a.mean()),a.std())
np.hstack((a,b,c,d))

f = np.array([[[1,-1,2,1]],[[1,-1,1,1]]])
f

array([[[ 1, -1,  2,  1]],

       [[ 1, -1,  1,  1]]])

In [14]:
#实验的CELL
dataset1 = ImageFolder("D:\\newfolder43_1")
img = dataset1[0][0]
image = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
cv2.imshow("OpenCV",image)  
cv2.waitKey() 

-1

In [19]:
#实验的CELL
image = cv2.imread("D:\\newfolder43_1\\clear_face\\picture_3144_.png")
#image = image[:,:,0]
print(image.var())
image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
print(image.var())
cv2.imshow("OpenCV",image)  
cv2.waitKey() 

1728.6520611234355
1728.6520611234355


-1

In [26]:
#实验的CELL
normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])   #感觉不应该进行方差标准化，因为最终图像就是使用方差来完成模糊程度判断的。
transform0 = transforms.Compose([
                                #transforms.RandomRotation(20),         #图像旋转的角度
                                #transforms.RandomHorizontalFlip(),    #水平方向翻转
                                transforms.ToTensor(),
                                normalize()
                                ])
dataset = ImageFolder("D:\\newfolder43_1",transform=transform0)

TypeError: __call__() missing 1 required positional argument: 'tensor'

In [5]:
batch_size = 64
learning_rate = 1e-4
num_epoches = 300
weight_decay = 1e-4

In [6]:
dataset0 = var_dataset("D:\\newfolder43_1")
dataset0[0][1]


0

In [7]:
dataloader = DataLoader(dataset0,batch_size=batch_size,shuffle=True)
#dataiter = iter(dataloader)
#imgs,labels = next(dataiter)
#labels

In [8]:
input_dim=224
n_hidden_1=64
n_hidden_2=32
n_hidden_3=16
output_dim=2
model = FC_Net(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)

In [9]:
if t.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
criterion.cuda() 
optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay) 

In [ ]:
print("training start time  <===>  ",time.localtime())
sum_loss = t.Tensor([0])
for epoch in range(0,num_epoches):
    for i,data in enumerate(dataloader,0):
        data_batch, label = data        
        data_batch, label = Variable(data_batch.float()).cuda(),Variable(label).cuda()
        output = model(data_batch)
        optimizer.zero_grad()
        loss = criterion(output,label)
        loss.backward() 
        optimizer.step()
        
        sum_loss = t.cat((sum_loss,loss.cpu().data),0)
        x = i+epoch*len(dataloader)
        if x%50 == 49:
            loss_aver = sum_loss.mean()
            sum_loss = t.Tensor([0])
            vis.line(X=t.Tensor([x]), Y=t.Tensor([loss_aver]), win='blur2',opts={'title': 'loss-batch1', 'xlabel':'batch_num','ylabel':'loss_aver'},update='append' if x>49 else None)
            #vis.updateTrace(X=t.Tensor([x]),Y=t.Tensor([loss_aver]),win='face_angle', name='7')
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\grad_var_full_layers\\model_save\\fc_network_300_64_32_NoSoftmax_224_guiyihua_0.5Normalize_lr1e-4.pth") 
print("training start time  <===>  ",time.localtime())                

training start time  <===>   time.struct_time(tm_year=2018, tm_mon=6, tm_mday=13, tm_hour=20, tm_min=37, tm_sec=29, tm_wday=2, tm_yday=164, tm_isdst=0)


#以下是测试部分

In [14]:
from PIL import Image
import shutil

model.cuda()
model.eval()

test_right_num = 0
test_right_num = t.Tensor([test_right_num]).int()
test_right_num = Variable(test_right_num)

test_dataset = var_dataset("D:\\newfolder43_2")
for img0, label0 in test_dataset:
    img0 = Variable(img0).view(1,224).cuda()
    label0 = Variable(t.Tensor([label0]).long()).cuda()
    output = model(img0)
    
    _,pred = t.max(output.cpu(),1)
    num_correct = (pred==label0.cpu()).sum()
    num_correct = num_correct.int()
    test_right_num = test_right_num + num_correct
    
acc = (float(test_right_num))/len(test_dataset)
print(test_right_num)
print(len(test_dataset))
print(acc)    
    

Variable containing:
 2854
[torch.IntTensor of size 1]

3172
0.8997477931904161
